In [1]:
import pandas as pd


move= pd.read_csv(r"C:\Users\ywan3\Desktop\Economics 690\Machine learning\Project\upcber.csv")
move.head()

,COM_CODE,UPC,DESCRIP,SIZE,CASE,NITEM
0,27,294,BEER LIMIT,12/12O,2,9990180
1,26,307,HEINEKEN KINGSIZE CA,259 OZ,1,9490410
2,27,710,BUDWEISER BEER,24/12O,1,9402300
3,27,711,BUDWEISER DRY BEER,24/12O,1,9402650
4,27,712,BUDWEISER LIGHT BEER,24/12O,1,9402600


In [2]:
beer_brand=move[["DESCRIP"]]
beer_brand

,DESCRIP
0,BEER LIMIT
1,HEINEKEN KINGSIZE CA
2,BUDWEISER BEER
3,BUDWEISER DRY BEER
4,BUDWEISER LIGHT BEER
5,COORS BEER
6,COORS EXTRA GOLD BEE
7,KEYSTONE REGULAR BEE
8,MILLER HIGH LIFE PAR
9,MILLER LITE BEER


In [3]:
import nltk

In [4]:
try:
    import nltk

    stemmer = nltk.PorterStemmer()
    for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
        print(word, "=>", stemmer.stem(word))
except ImportError:
    print("Error: stemming requires the NLTK module.")
    stemmer = None

Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
import re
from collections import Counter
import numpy as np
class WordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, lower_case=True, remove_punctuation=True,stemming=True):
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.stemming = stemming
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for beer in X:
            text = beer or ""
            if self.lower_case:
                text = text.lower()
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word) # if len(word)>5 else word
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [6]:
X_few = beer_brand[:3].values.reshape(-1)
X_few_wordcounts = WordCounterTransformer().fit_transform(X_few)
X_few_wordcounts

array([Counter({'beer': 1, 'limit': 1}),
       Counter({'heineken': 1, 'kingsiz': 1, 'ca': 1}),
       Counter({'budweis': 1, 'beer': 1})], dtype=object)

In [7]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [8]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors

<3x11 sparse matrix of type '<class 'numpy.int32'>'
	with 7 stored elements in Compressed Sparse Row format>

In [9]:
X_few_vectors.toarray()

array([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]], dtype=int32)

In [10]:
X = beer_brand.values.reshape(-1)
X_wordcounts = WordCounterTransformer().fit_transform(X)
X_wordcounts[:5]

array([Counter({'beer': 1, 'limit': 1}),
       Counter({'heineken': 1, 'kingsiz': 1, 'ca': 1}),
       Counter({'budweis': 1, 'beer': 1}),
       Counter({'budweis': 1, 'dri': 1, 'beer': 1}),
       Counter({'budweis': 1, 'light': 1, 'beer': 1})], dtype=object)

In [11]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=1000)
#X_vectors = vocab_transformer.fit_transform(X_wordcounts)
#X_vectors
vocab_transformer.fit(X_wordcounts)

WordCounterToVectorTransformer(vocabulary_size=1000)

In [12]:
len(vocab_transformer.vocabulary_ )

467

In [13]:
word_count=pd.DataFrame(vocab_transformer.most_common_)

In [14]:
word_count.columns = word_count.columns.astype(str)
word_count=word_count.rename(index=str, columns={"0": "word", "1": "count"})

In [15]:
count_gt1= word_count[word_count["count"]>1]
count_eq1= word_count[word_count["count"]==1]

In [16]:
move.dropna(inplace = True) 
  
# new data frame with split value columns 
new = move["SIZE"].str.split("/", n = 1, expand = True) 
  
# making seperate first name column from new data frame 
move["size"]= new[0] 
  
# making seperate last name column from new data frame 
move["volume"]= new[1] 

In [17]:
move.head()

,COM_CODE,UPC,DESCRIP,SIZE,CASE,NITEM,size,volume
0,27,294,BEER LIMIT,12/12O,2,9990180,12,12O
1,26,307,HEINEKEN KINGSIZE CA,259 OZ,1,9490410,259 OZ,None
2,27,710,BUDWEISER BEER,24/12O,1,9402300,24,12O
3,27,711,BUDWEISER DRY BEER,24/12O,1,9402650,24,12O
4,27,712,BUDWEISER LIGHT BEER,24/12O,1,9402600,24,12O


In [18]:
move= move.fillna(0)
idx = (move['volume'] == 0)
move.loc[idx,['size','volume']] = move.loc[idx,['volume','size']].values
move

,COM_CODE,UPC,DESCRIP,SIZE,CASE,NITEM,size,volume
0,27,294,BEER LIMIT,12/12O,2,9990180,12,12O
1,26,307,HEINEKEN KINGSIZE CA,259 OZ,1,9490410,0,259 OZ
2,27,710,BUDWEISER BEER,24/12O,1,9402300,24,12O
3,27,711,BUDWEISER DRY BEER,24/12O,1,9402650,24,12O
4,27,712,BUDWEISER LIGHT BEER,24/12O,1,9402600,24,12O
5,27,720,COORS BEER,24/12O,1,9410150,24,12O
6,27,721,COORS EXTRA GOLD BEE,24/12O,1,9410230,24,12O
7,27,723,KEYSTONE REGULAR BEE,24/12O,1,9428700,24,12O
8,27,731,MILLER HIGH LIFE PAR,30/12O,1,9451440,30,12O
9,27,732,MILLER LITE BEER,24/12O,1,9451200,24,12O


In [19]:
move['size']=move['size'].replace({0: None})
move.head()

,COM_CODE,UPC,DESCRIP,SIZE,CASE,NITEM,size,volume
0,27,294,BEER LIMIT,12/12O,2,9990180,12,12O
1,26,307,HEINEKEN KINGSIZE CA,259 OZ,1,9490410,None,259 OZ
2,27,710,BUDWEISER BEER,24/12O,1,9402300,24,12O
3,27,711,BUDWEISER DRY BEER,24/12O,1,9402650,24,12O
4,27,712,BUDWEISER LIGHT BEER,24/12O,1,9402600,24,12O


In [20]:
writer = pd.ExcelWriter('new_move.xlsx')
move.to_excel(writer,'Sheet1')
writer.save()

In [21]:
eq1= pd.read_excel(r"C:\Users\ywan3\Desktop\Economics 690\Machine learning\Project\count_eq1.xlsx")
gt1= pd.read_excel(r"C:\Users\ywan3\Desktop\Economics 690\Machine learning\Project\count_gt1.xlsx")


In [22]:
new_merge= pd.concat([gt1,eq1])["word"]

C:\Users\ywan3\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [23]:
dic = dict( (w,i+1)   for i,w in enumerate(new_merge.values))
dic    

{'light': 1,
 'old': 2,
 'miller': 3,
 'ale': 4,
 'budweis': 5,
 'draft': 6,
 'coor': 7,
 'ice': 8,
 'red': 9,
 'bee': 10,
 'milwauke': 11,
 'michelob': 12,
 'lager': 13,
 'adam': 14,
 'special': 15,
 'export': 16,
 'dri': 17,
 'stroh': 18,
 'samuel': 19,
 'classic': 20,
 'dark': 21,
 'golden': 22,
 'lite': 23,
 'blue': 24,
 'genuin': 25,
 'augsburg': 26,
 'dr': 27,
 'pale': 28,
 'pete': 29,
 'high': 30,
 'life': 31,
 'leinenkugel': 32,
 'amber': 33,
 'regular': 34,
 'draf': 35,
 'st': 36,
 'extra': 37,
 'island': 38,
 'beck': 39,
 'wick': 40,
 'lnnr': 41,
 'drft': 42,
 'lt': 43,
 'stout': 44,
 'hamm': 45,
 'honey': 46,
 'bock': 47,
 'goos': 48,
 'molson': 49,
 'oregon': 50,
 'gold': 51,
 'busch': 52,
 'long': 53,
 'dog': 54,
 'lowenbrau': 55,
 'tecat': 56,
 'big': 57,
 'al': 58,
 'icehous': 59,
 'be': 60,
 'irish': 61,
 'murphi': 62,
 'longnec': 63,
 'longn': 64,
 'pabst': 65,
 'malt': 66,
 'black': 67,
 'schlitz': 68,
 'labatt': 69,
 'foster': 70,
 'moon': 71,
 'killian': 72,
 'blk':

In [24]:
vocab_transformer.vocabulary_ = dic 

In [25]:
want=pd.DataFrame(vocab_transformer.transform(X_wordcounts).toarray()[:,:398])
want

,0,1,2,3,4,5,6,7,8,9,...,388,389,390,391,392,393,394,395,396,397
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
for x in range(0,398,1):
    want[x].loc[(want[x] > 1)] = 1


In [27]:
want1=want

In [28]:
res = dict((v,k) for k,v in dic.items())
res

{1: 'light',
 2: 'old',
 3: 'miller',
 4: 'ale',
 5: 'budweis',
 6: 'draft',
 7: 'coor',
 8: 'ice',
 9: 'red',
 10: 'bee',
 11: 'milwauke',
 12: 'michelob',
 13: 'lager',
 14: 'adam',
 15: 'special',
 16: 'export',
 17: 'dri',
 18: 'stroh',
 19: 'samuel',
 20: 'classic',
 21: 'dark',
 22: 'golden',
 23: 'lite',
 24: 'blue',
 25: 'genuin',
 26: 'augsburg',
 27: 'dr',
 28: 'pale',
 29: 'pete',
 30: 'high',
 31: 'life',
 32: 'leinenkugel',
 33: 'amber',
 34: 'regular',
 35: 'draf',
 36: 'st',
 37: 'extra',
 38: 'island',
 39: 'beck',
 40: 'wick',
 41: 'lnnr',
 42: 'drft',
 43: 'lt',
 44: 'stout',
 45: 'hamm',
 46: 'honey',
 47: 'bock',
 48: 'goos',
 49: 'molson',
 50: 'oregon',
 51: 'gold',
 52: 'busch',
 53: 'long',
 54: 'dog',
 55: 'lowenbrau',
 56: 'tecat',
 57: 'big',
 58: 'al',
 59: 'icehous',
 60: 'be',
 61: 'irish',
 62: 'murphi',
 63: 'longnec',
 64: 'longn',
 65: 'pabst',
 66: 'malt',
 67: 'black',
 68: 'schlitz',
 69: 'labatt',
 70: 'foster',
 71: 'moon',
 72: 'killian',
 73: 'b

In [29]:
want1= want1.rename(columns=res)
want1= want1.rename(columns={0:'other'})
want1.head()

,other,light,old,miller,ale,budweis,draft,coor,ice,red,...,honker,hexnut,midnigh,dortmund,kozel,czech,fin,thumper,blackberri,summer
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
writer1 = pd.ExcelWriter('dummy.xlsx')
want1.to_excel(writer1,'Sheet1')
writer1.save()

In [31]:
wber= pd.read_csv(r"C:\Users\ywan3\Desktop\Economics 690\Machine learning\Project\wber.csv")
wber.head()

,STORE,UPC,WEEK,MOVE,QTY,PRICE,SALE,PROFIT,OK,PRICE_HEX,PROFIT_HEX
0,2,294,298,11,1,2.62,NaN,-18.83,1,4004F5C28F5C28F6,C032D47AE147AE14
1,2,294,299,9,1,2.51,NaN,18.47,1,4004147AE147AE14,40327851EB851EB8
2,2,294,300,6,1,2.49,NaN,13.60,1,4003EB851EB851EC,402B333333333333
3,2,294,301,5,1,2.99,NaN,-67.89,1,4007EB851EB851EC,C050F8F5C28F5C29
4,2,294,302,1,1,2.49,NaN,13.60,1,4003EB851EB851EC,402B333333333333


In [32]:
move= pd.read_excel(r"C:\Users\ywan3\Desktop\Economics 690\Machine learning\Project\new_move.xlsx")
move.head()

,COM_CODE,UPC,DESCRIP,SIZE,CASE,NITEM,size,volume(oz),other,light,...,honker,hexnut,midnigh,dortmund,kozel,czech,fin,thumper,blackberri,summer
0,27,294,BEER LIMIT,12/12O,2,9990180,12.0,12.0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,26,307,HEINEKEN KINGSIZE CA,259 OZ,1,9490410,NaN,259.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27,710,BUDWEISER BEER,24/12O,1,9402300,24.0,12.0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,27,711,BUDWEISER DRY BEER,24/12O,1,9402650,24.0,12.0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,27,712,BUDWEISER LIGHT BEER,24/12O,1,9402600,24.0,12.0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [34]:
new= pd.merge(wber,move,on="UPC")
new.head()

MemoryError: 